In [1]:
## Import Some Libraries
import json
import numpy as np

In [19]:
class LoadModel:
    def __init__(self, file_path):
        with open(file_path, 'r') as f:
            model_data = json.load(f)
        
        layers = model_data['layers']
        n_layers = len(layers)
        
        self.weights = []
        self.biases = []
        self.activations = []
        self.sigmoid_gradient = lambda x: x * (1 - x)
        self.relu_gradient = lambda x: 1. * (x > 0)
        self.tanh_gradient = lambda x: 1 - np.tanh(x)**2
        self.softmax_gradient = lambda x: x * (1 - x)
        
        for i in range(1, n_layers):
            layer = layers[i]
            
            if layer['tipe'] == 'hidden':
                n_neurons = layer['n_neuron']
                weight = np.array(layer['weight'])
                bias = np.array(layer['bias'])
                activation = layer['activation_function']
                
                self.weights.append(weight)
                self.biases.append(bias)
                self.activations.append(activation)
                
            elif layer['tipe'] == 'output':
                n_neurons = layer['n_neuron']
                weight = np.array(layer['weight'])
                bias = np.array(layer['bias'])
                activation = layer['activation']
                
                self.weights.append(weight)
                self.biases.append(bias)
                self.activations.append(activation)
                
    def get_weights(self):
        return self.weights
    
    def get_biases(self):
        return self.biases
    
    def get_activations(self):
        return self.activations
    
    def get_layer(self, layer):
        return self.weights[layer], self.biases[layer], self.activations[layer]
    
    def get_layer_weights(self, layer):
        return self.weights[layer]
    
    def get_layer_biases(self, layer):
        return self.biases[layer]
    
    def get_layer_activations(self, layer):
        return self.activations[layer]
    
    def get_model(self):
        return self.weights, self.biases, self.activations
    
    def get_activation_gradient(self, activation):
        if activation == 'relu':
            return lambda x: 1. * (x > 0)
        elif activation == 'sigmoid':
            return lambda x: x * (1. - x)
        elif activation == 'linear':
            return lambda x: np.ones_like(x)
        elif activation == 'softmax':
            return lambda x: x * (1. - x)

    
    # print model per layer
    def print_model(self):
        for i in range(len(self.weights)):
            print('Layer', i+1)
            print('Weights:', self.weights[i])
            print('Biases:', self.biases[i])
            print('Activations:', self.activations[i])
            print('')

In [26]:
class BackPropagation:
    def __init__(self, model_path):
        self.model = LoadModel(model_path)
        self.weights = self.model.get_weights()
        self.biases = self.model.get_biases()
        self.activations = self.model.get_activations()
        self.sigmoid_gradient = lambda x: x * (1 - x)
        self.relu_gradient = lambda x: 1. * (x > 0)
        self.tanh_gradient = lambda x: 1 - np.tanh(x)**2
        self.softmax_gradient = lambda x: x * (1 - x)

    def get_activation_gradient(self, activation):
        if activation == 'relu':
            return lambda x: 1. * (x > 0)
        elif activation == 'sigmoid':
            return lambda x: x * (1. - x)
        elif activation == 'linear':
            return lambda x: np.ones_like(x)
        elif activation == 'softmax':
            return lambda x: x * (1. - x)
        
    def feed_forward(self, input_data):
        input_data = np.array(input_data)
        n_layers = len(self.weights)
        
        self.z = []
        self.a = []
        
        self.a.append(input_data)
        
        for i in range(n_layers):
            z = np.dot(self.weights[i], self.a[i]) + self.biases[i]
            self.z.append(z)
            
            if i == n_layers-1:
                a = self.softmax(z)
            else:
                a = self.relu(z)
                
            self.a.append(a)
            
        return self.a[-1]
    
    def relu(self, x):
        return np.maximum(x, 0)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=0)
    
    def tanh(self, x):
        return np.tanh(x)
    
    def back_propagation(self, input_data, target):
        input_data = np.array(input_data)
        target = np.array(target)
        
        n_layers = len(self.weights)
        
        self.feed_forward(input_data)
        
        delta = []
        nabla_w = []
        nabla_b = []
        
        # output layer
        delta.append(self.a[-1] - target)
        nabla_w.append(np.dot(delta[-1], self.a[-2].T))
        nabla_b.append(delta[-1])
        
        # hidden layer
        for i in range(n_layers-2, -1, -1):
            delta.append(np.dot(self.weights[i+1].T, delta[-1]) * self.relu_gradient(self.a[i+1]))
            nabla_w.append(np.dot(delta[-1], self.a[i].T))
            nabla_b.append(delta[-1])
            
        nabla_w.reverse()
        nabla_b.reverse()
        
        return nabla_w, nabla_b
    
    def update_weights(self, nabla_w, nabla_b, learning_rate):
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * nabla_w[i]
            self.biases[i] -= learning_rate * nabla_b[i]

def main():
    model_path = 'relu.json'
    backprop = BackPropagation(model_path)
    
    input_data = np.array([[0,0], [0,1], [1,0], [1,1]])
    target = [0, 1, 1, 0]
    
    print('Feed Forward')
    print(backprop.feed_forward(input_data))
    print('')
    
    print('Back Propagation')
    nabla_w, nabla_b = backprop.back_propagation(input_data, target)
    print('Nabla W:', nabla_w)
    print('Nabla B:', nabla_b)
    print('')
    
    print('Update Weights')
    backprop.update_weights(nabla_w, nabla_b, 0.1)
    print('Weights:', backprop.weights)
    print('Biases:', backprop.biases)

if __name__ == '__main__':
    main()

Feed Forward


ValueError: shapes (2,2) and (4,2) not aligned: 2 (dim 1) != 4 (dim 0)